In [ ]:
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import Select
from time import sleep
import csv
import numpy as np
import time

In [ ]:
chromedriver = ''  # your chromedriver's dir
twseStock = 'http://www.twse.com.tw/zh/page/trading/exchange/STOCK_DAY_AVG.html'
driver = webdriver.Chrome(chromedriver)
driver.get(twseStock) # open twseStock website

In [ ]:
selectStockList = driver.find_element_by_class_name('stock-code-browse')
selectStockList.click()

In [ ]:
driverSource = driver.page_source
dataSource = BeautifulSoup(driverSource, 'lxml')

In [ ]:
# 證券分類 stk
stkDomain_str = dataSource.find('div', class_='category stk').find_all('a')
stkDomain_arr = []
for i in stkDomain_str:
#     print(i.get_text())
    stkDomain_arr.append(i.get_text())
print(stkDomain_arr)

In [ ]:
# targetDomain = stkDomain_arr[1]
driver.find_element_by_link_text('化學工業').click()

In [ ]:
driverSource = driver.page_source
soup = BeautifulSoup(driverSource, 'lxml')

In [ ]:
divAll = soup.find('div', class_='items').find_all('a')
stockList =[]
for i in divAll: 
    stockList.append(i.string) # 分類的基金清單
print(stockList)

In [ ]:
stockList = ['1730', '1723', '1724', '1713', '1711', '1727', '1704', '1721', '1708', '1725']
thisTimeStock = stockList[0] # choose your target index

In [ ]:
for stock in stockList:
    stockPriceList =[]
    
    # 先點回到首頁http://www.twse.com.tw/zh/page/trading/exchange/STOCK_DAY_AVG.html
    for year in range(2017, 2018):
        # 填年
        selectYear = Select(driver.find_element_by_name('yy'))
        selectYear.select_by_value(str(year))
        
        for month in range(1, 13):
            # 填月
            selectMonth = Select(driver.find_element_by_name('mm'))
            selectMonth.select_by_value(str(month))

            # 填股票代碼
            driver.find_element_by_name('stockNo').clear()
            driver.find_element_by_name('stockNo').send_keys(stock)
            driver.find_element_by_css_selector('.button.search').click()
            
            time.sleep(5) # wait
            
            driverSource = driver.page_source
            stockSoup = BeautifulSoup(driverSource, 'lxml')
            try:
                stock_Date_Close = stockSoup.find(id='report-table').find('tbody').find_all('td')
                for i in stock_Date_Close:
                    print(i.get_text())
                    
                    tmp = i.get_text()
                    stockPriceList.append(tmp)
            except:
                print('No data')

In [ ]:
# show month avg closed
avgClosed = []
for i in range(0, len(stockPriceList)):
    if stockPriceList[i] == '月平均收盤價':
        avgClosed.append(stockPriceList[i+1])
        print(stockPriceList[i+1])
# print(avgClosed)

In [ ]:
# change to dict
stockPriceList_dict = {} 
for i in range(0, len(stockPriceList), 2):
    stockPriceList_dict[stockPriceList[i]] = stockPriceList[i+1]

# save to csv file
fileName = '/' + stock + '.csv' # your file name
pathProg = '' # save csv file at where

with open(pathProg + fileName, 'w') as csv:
    columnTitleRow = "date, closedPrice\n"
    csv.write(columnTitleRow)

    for key in stockPriceList_dict.keys():
        date = key
        closedPrice = stockPriceList_dict[key]
        row = date + "," + closedPrice + "\n"
        csv.write(row)